In [28]:
import json
import csv
import os

In [29]:
jsonDataFiles = []
for dirs, subdirs, files in os.walk("../data/zipHistory/"):
    for file in files:
        if file.endswith(".json"):
            jsonDataFiles.append(os.path.join(dirs, file))

In [30]:
zip2County = {}
with open("../data./zip2County.csv") as f:
    reader = csv.reader(f)
    zip2County = {rows[0]:rows[1] for rows in reader}

In [31]:
import math

def F2C(temp):
    return (temp - 32) * 5/9

def getRH(t,d):
    return 100 *(math.exp((17.625*d)/(243.04+d))/(math.exp((17.625*t)/(243.04+t))))

countyHistory = {}
countyHistoryDewPoint = {}
countryRH = {}
for jsonFile in jsonDataFiles:
    with open(jsonFile, 'r') as f:
        zipcode = jsonFile.split('/')[-1].split('.')[0]
        averageTemp = 0
        averageDewPoint = 0
        averageRH = 0
        numDataPointsTemp = 0
        numDataPointsDewPoint = 0
        numDataPointsRH = 0
        data = json.load(f)
        for year in data:
            for month in data[year]:
                try:
                    temp = 0
                    if "High Temp:" in month and month["High Temp:"][:3].replace('Â','').replace('°','') != '':
                        temp = float(month["High Temp:"][:3].replace('Â','').replace('°',''))
                        averageTemp += temp
                        numDataPointsTemp += 1
                    if "Dewpoint:" in month and month["Dewpoint:"][:-3].replace('Â','').replace('°',''):
                        dewpoint = float(month["Dewpoint:"][:-3].replace('Â','').replace('°',''))
                        averageDewPoint += dewpoint
                        numDataPointsDewPoint += 1

                    if temp != 0 and dewpoint != 0:
                        rh = getRH(F2C(temp), F2C(dewpoint))
                        averageRH += rh
                        numDataPointsRH += 1
                except:
                    pass
        if numDataPointsTemp > 0:
            averageTemp /= numDataPointsTemp
        if numDataPointsDewPoint > 0:
            averageDewPoint /= numDataPointsDewPoint
        if numDataPointsRH > 0:
            averageRH = averageRH / numDataPointsRH

        if (zipcode != ''):
            if(zipcode in zip2County):
                if zip2County[zipcode] in countyHistory:
                    countyHistory[zip2County[zipcode]].append(averageTemp)
                else:
                    countyHistory[zip2County[zipcode]] = [averageTemp]

                if zip2County[zipcode] in countyHistoryDewPoint:
                    countyHistoryDewPoint[zip2County[zipcode]].append(averageDewPoint)
                else:
                    countyHistoryDewPoint[zip2County[zipcode]] = [averageDewPoint]

                if zip2County[zipcode] in countryRH:
                    countryRH[zip2County[zipcode]].append(averageRH)
                else:
                    countryRH[zip2County[zipcode]] = [averageRH]

In [30]:
with open("highTempCounty.csv",'w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["County","High Temp"])
    for county in countyHistory:
        temp =sum(countyHistory[county])/len(countyHistory[county])
        writer.writerow([county, temp])

In [24]:
with open("dewPointCounty.csv",'w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["County","Dew Point"])
    for county in countyHistoryDewPoint:
        temp =sum(countyHistoryDewPoint[county])/len(countyHistoryDewPoint[county])
        writer.writerow([county, temp])

In [32]:
with open("RHCounty.csv",'w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["County","RH"])
    for county in countryRH:
        temp =sum(countryRH[county])/len(countryRH[county])
        writer.writerow([county, temp])